In [ ]:
import datetime
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path

from summer.utils import ref_times_to_dti

from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.runs import ManagedRun
from autumn.core.project import get_project

In [ ]:
region = "northern_territory"

In [ ]:
run_id = "sm_sir/northern_territory/1662380460/28ff0b7"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
pbi = mr.powerbi.get_db()
results = pbi.get_uncertainty()

In [ ]:
project = get_project("sm_sir", region, reload=True)

In [ ]:
title_lookup = {
    "notifications": "daily notifications",
    "infection_deaths": "COVID-19-specific deaths",
    "hospital_admissions": "new daily hospital admissions",
    "icu_admissions": "new daily admissions to ICU",
    "proportion_seropositive": "proportion recovered from COVID-19",
    "incidence": "daily new infections",
    "prop_incidence_strain_delta": "proportion of cases due to Delta",
    "hospital_admissions": "daily hospital admissions",
    "hospital_occupancy": "total hospital beds",
    "icu_admissions": "daily ICU admissions",
    "icu_occupancy": "total ICU beds",
    "prop_ever_infected": "ever infected with Delta or Omicron",
    "cumulative_infection_deaths": "cumulative COVID-19 deaths",
    "cumulative_hospital_admissions": "cumulative hospital admissions",
    "cumulative_icu_admissions": "cumulative ICU admissions",    
}

In [ ]:
def outputs_with_uncertainty(outputs, scenarios, input_shape=None):
    
    # Set row and col position for outputs based on requested outputs
    # if use defines own shape/layout
    if input_shape:
        shape = input_shape
    # Otherwise define shape based on requested outputs
    else:
        if len(outputs) == 4:
            shape = [2, 2]
        elif len(outputs) == 3:
            shape = [1, 3]
        elif len(outputs) == 2:
            shape = [1, 2]
        elif len(outputs) == 1:
            shape = [1, 1]
    
    # Store number of rows and columns
    rows = shape[0]
    cols = shape[1]
    
    # Create all [row, col] positions
    positions = []
    for r in range(rows):
        for c in range(cols):
            positions.append([r+1, c+1])
    
    
    fig = make_subplots(rows, cols, subplot_titles=outputs)
    
    # Define colours for scenarios as partial rgba strings to allow for specifying opacity with alpha
    colours = {0: 'rgba(26,150,65,',
               1: 'rgba(248,156,116,',
               2: 'rgba(204, 102, 119,',
               3: 'rgba(29, 105, 150,',
               4: 'rgba(221, 204, 119,'
              }
    
    for pos, o in zip(positions, outputs):
        #output_row = 1 if outputs.index(o) == 0 or outputs.index(o) == 1 else 2
        #output_col = 1 if outputs.index(o) == 0 or outputs.index(o) == 3 else 2
        for s in scenarios:
            colour = colours[s]

            
            # Set transparency
            median_alpha = 1.0
            fifty_cred_alpha = 0.4
            ninety_five_cred_alpha = 0.2
            
            # Set data, index and labels
            results_df = results[(o, s)]
            indices = results_df.index
            label = "baseline" if s == 0 else project.param_set.scenarios[s - 1]["description"]
            show_legend = True if pos[0] == 1 and pos[1] == 1 else False
            
            # add median trace
            fig.add_trace(go.Scatter(x=indices, y=results_df[0.500], 
                                     line=dict(color=(colour+str(median_alpha)+")")), 
                                     name=label,
                                     showlegend = show_legend
                                    ),
                         row = pos[0],
                          col= pos[1]
                         )
            
            #50% CI trace
            fig.add_traces([go.Scatter(x=results_df.index, y=results_df[0.250], 
                                       line=dict(color=colour+str(fifty_cred_alpha)+")"),
                                       fillcolor=colour+str(fifty_cred_alpha)+")", showlegend=False),
                            go.Scatter(x=results_df.index, y=results_df[0.750], 
                                       line=dict(color=colour+str(fifty_cred_alpha)+")"), 
                                       fillcolor=colour+str(fifty_cred_alpha)+")", 
                                       fill="tonexty", showlegend=False)
                           ],
                           rows = pos[0],
                          cols = pos[1]
                          )
            
            #95% CI trace
            fig.add_traces([go.Scatter(x=results_df.index, y=results_df[0.025], 
                                       line=dict(color=colour+str(ninety_five_cred_alpha)+")"), 
                                       fillcolor=colour+str(ninety_five_cred_alpha)+")", showlegend=False),
                            go.Scatter(x=results_df.index, y=results_df[0.975], 
                                       line=dict(color=colour+str(ninety_five_cred_alpha)+")"),
                                       fillcolor=colour+str(ninety_five_cred_alpha)+")", 
                                       fill="tonexty", showlegend=False)
                           ],
                           rows = pos[0],
                          cols = pos[1]
                          )
    # subplot titles stored as annotations;
    #code pinched from user vestland: 
    #https://stackoverflow.com/questions/63220009/how-do-i-set-each-plotly-subplot-title-during-graph-creation-loop
    fig.for_each_annotation(lambda a: a.update(text = title_lookup[a.text]))
    
    # place legend in upper left hand corner
    fig.update_layout(legend_x=-0.05, legend_y=1.3,
                      # Need to adjust height and width
                     height=400, width=800)
    
    # Update image config to allow for higher resolution image to be quick downloaded
    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': 'image_1',
            'height': 400,
            'width': 1000,
            'scale':4 #adjust scale to increase resolution
        }
    }


    fig.show(config=config)
    
   

In [ ]:
# Specify scenarios here
scenarios = [0, 4]

# Specify outputs here up to 4: function will determine shape based on number of outputs specified
outputs = ('notifications',) #"hospital_occupancy",) # 'icu_admissions', "infection_deaths")

outputs_with_uncertainty(outputs, scenarios)